<a href="https://colab.research.google.com/github/KenzioDG/Colab-Projects/blob/main/BERT_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Analisis dataset ini bertujuan untuk mendapatkan komparasi model klasifikasi data text yang berupa deskripsi produk dari website e-commerce, yang terdiri dari emoat label: _Household, Books, Electronics, dan Clothing & Accessories_.
- Pada tahap awal, dataset akan melalui proses pre-processing untuk mengolah teks deskripsi menjadi bentuk dasar sesuai dengan tata bahasa standar. Proses ini akan mencakup langkah-langkah seperti penghapusan karakter non-alfanumerik, tokenisasi, penghapusan kata-kata yang tidak relevan (stop words), dan lemmatisasi untuk mengubah kata-kata ke dalam bentuk dasar.

- Setelah pre-processing, dilakukan vectorization untuk mengubah teks menjadi representasi numerik yang dapat digunakan oleh algoritma machine learning.

- Kemudian dibandingkan dua metode vectorization, yaitu Bag of Words dan TF-IDF, serta dua algoritma machine learning, yaitu Naive Bayes dan Random Forest Classifier. Dengan membandingkan hasil dan performa dari kombinasi vectorizer dan model klasifikasi yang berbeda, maka dapat didapatkan komparasi kemampuan dan kecocokan metode algoritma yang digunakan dalam melakukan klasifikasi pada dataset ini.

In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import gdown
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


Dengan library gdown, data dari link tautan Gdrive yang terbuka dapat langsung diunduh dan masuk ke dalam runtime colab. Hanya dibutuhkan file ID dari file yang ingin diunduh

In [ ]:
!gdown 16l1cSKGFrWahHq-ylAiPRQQuihfM5qZi

Downloading...
From: https://drive.google.com/uc?id=16l1cSKGFrWahHq-ylAiPRQQuihfM5qZi
To: /content/data_1A.csv
100% 9.28M/9.28M [00:00<00:00, 91.2MB/s]


Pada kode di bawah ini, data dibaca dengan library pandas agar terformat dengan benar. Selanjutnya kolom teks pada dataset diubah tipe datanya sebagai string, karena tanpa perubahan manual ini biasanya ada masalah ketidakcocokan tipe data pada tahap-tahap selanjutnya

In [ ]:
data = pd.read_csv('data_1A.csv')
print(data.head())
print(data['label'].value_counts())
data['text'] = data['text'].astype(str)


   Unnamed: 0                                               text      label
0           0  The Theory of Everything Review Stephen Hawkin...      Books
1           1  Computer Networks: A Top - Down Approach About...      Books
2           2  Sajani Premium Quality Brown Wooden Coat Hange...  Household
3           3  Bosch Lifestyle MCM3501M 800-Watt Food Process...  Household
4           4  Secret Wish Women's Navy-Blue Towel Bathrobe (...  Household
Household                 4832
Books                     2951
Electronics               2643
Clothing & Accessories    2180
Name: label, dtype: int64


## Text Preprocessing

### Cleaning & Lemmatization

Kemudiab di bawah akan dilakukan pengunduhan dari dua resource NLP, yaitu Wordnet dan punkt.

**WordNet**


---



---


WordNet menyediakan informasi tentang makna kata, hubungan antara kata (misalnya, sinonim, antonim, hiperonim, hiponim), dan pengelompokan kata ke dalam kelompok semantik yang disebut "synset" (synonym set). NLTK menggunakan WordNet sebagai salah satu sumber daya untuk melakukan lemmatisasi, yaitu mengubah kata ke dalam bentuk dasarnya. Lemmatisasi berguna untuk mengurangi variasi kata ke dalam bentuk yang standar agar dapat diolah lebih efisien.

**Punkt**


---



---


Punkt adalah model yang dilatih untuk melakukan tokenisasi pada teks bahasa Inggris. Tokenisasi adalah proses memecah teks menjadi unit-unit yang lebih kecil, seperti kata-kata atau kalimat. Model Punkt yang disediakan oleh NLTK adalah model yang dilatih secara khusus untuk teks bahasa Inggris. Punkt digunakan oleh NLTK untuk memecah teks menjadi token-token yang lebih kecil sehingga dapat diolah dengan lebih baik, misalnya dalam pemrosesan teks, analisis kata, dan langkah-langkah lainnya yang melibatkan pemisahan teks menjadi bagian-bagian yang lebih terdefinisi.

---



---


Dengan mengunduh WordNet dan Punkt melalui NLTK, kita memiliki akses ke dua sumber daya penting yang membantu dalam pemrosesan bahasa alami, seperti lemmatisasi kata dan tokenisasi teks. Hal ini memudahkan kita untuk melakukan tahap-tahap preprocessing pada teks, seperti mengubah kata ke dalam bentuk dasarnya dan memecah teks menjadi unit-unit yang lebih kecil yang dapat diolah lebih lanjut.

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Lalu langsung saja dibuat fungsi untuk melakukan pembersihan text dengan beberapa parameter:

`[^\w\s]`: </br>
Ini adalah karakter set yang menunjukkan karakter apa yang akan dihapus. [^\w\s] berarti semua karakter yang bukan alfanumerik (\w) dan bukan spasi (\s) akan dihapus. Dengan ini, karakter khusus seperti tanda baca, simbol, dan karakter khusus lainnya akan dihapus dari teks.

`http[s]?://\S+`:</br>
Ini adalah pola yang digunakan untuk mendeteksi URL dalam teks. Pada pola ini, ada pencocokkan string yang dimulai dengan http:// atau https://, kemudian diikuti oleh satu atau lebih karakter yang bukan spasi (\S+). Dengan ini, URL yang terdapat dalam teks akan dihapus.

`''`:</br>
Ini adalah string kosong yang digunakan sebagai pengganti untuk karakter yang cocok dengan pola yang ditentukan. Dalam kasus ini, karakter yang cocok dengan pola akan dihapus dari teks.
</br>
Jadi, secara keseluruhan, dengan menggunakan re.sub(r'[^\w\s]|http[s]?://\S+', '', text), kita melakukan substitusi atau penggantian semua karakter khusus dan URL dalam teks dengan string kosong, sehingga menghapusnya dari teks.


---


Lali dibuat objek WordNetLemmatizer dari NLTK. WordNetLemmatizer digunakan untuk melakukan lemmatisasi kata dalam teks, yaitu mengubah kata-kata ke dalam bentuk dasarnya.



In [ ]:
# Download stopwords (hanya perlu dilakukan sekali)
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Menghapus karakter khusus
    text = re.sub(r'[^\w\s]|http[s]?://\S+', '', text)

    # Tokenisasi
    tokens = word_tokenize(text)

    # Mengubah teks menjadi lowercase
    tokens = [token.lower() for token in tokens]

    # Menghapus stopwords dan menerapkan lemmatization
    stop_words = set(stopwords.words('english'))
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]

    return ' '.join(tokens)

data['processed_text'] = data['text'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Maka di bawah ini adalah hasil teks setelah melalui fungsi preprocessing di atas

In [ ]:
data['processed_text'].head()

0    theory everything review stephen hawking theor...
1    computer network top approach author behrouz f...
2    sajani premium quality brown wooden coat hange...
3    bosch lifestyle mcm3501m 800watt food processo...
4    secret wish woman navyblue towel bathrobe free...
Name: processed_text, dtype: object

### Vectorization

Terdapat dua metode vectorisasi yang digunakan, yaitu Bag of Words (BoW) dan TF-IDF. Berikut adalah penjelasan singkat tentang kedua metode tersebut:

**Bag of Words (BoW):**


---


Bag of Words adalah metode vectorisasi yang sederhana. Pada metode ini, setiap kata dalam teks dianggap sebagai fitur dan dihitung frekuensi kemunculannya dalam setiap dokumen. Matriks BoW dibentuk dengan kolom-kolom yang merepresentasikan kata-kata unik dalam dataset, dan baris-baris yang merepresentasikan dokumen-dokumen atau sampel-sampel teks. Nilai dalam matriks ini menunjukkan jumlah kemunculan kata dalam setiap dokumen. Dengan menggunakan CountVectorizer dari library scikit-learn, kita dapat dengan mudah mengimplementasikan metode BoW.

**TF-IDF (Term Frequency-Inverse Document Frequency):**


---


TF-IDF adalah metode vectorisasi yang menggabungkan informasi tentang frekuensi kata dalam sebuah dokumen dengan informasi tentang seberapa umum kata tersebut dalam keseluruhan dataset. Ini membantu mengurangi bobot kata-kata yang umum dalam semua dokumen dan meningkatkan bobot kata-kata yang khusus untuk sebuah dokumen. TF-IDF menghitung dua nilai: Term Frequency (TF), yang menunjukkan frekuensi kemunculan kata dalam sebuah dokumen, dan Inverse Document Frequency (IDF), yang menunjukkan seberapa umum kata tersebut dalam semua dokumen.


---



---


Kedua metode ini menghasilkan representasi vektor numerik dari teks, di mana setiap dimensi dalam vektor ini merepresentasikan fitur kata-kata atau istilah-istilah dalam teks. Representasi vektor ini dapat digunakan sebagai input untuk algoritma machine learning seperti Naive Bayes atau Random Forest Classifier yang telah digunakan dalam kode tersebut. Dengan melakukan vectorization, teks menjadi bentuk yang dapat dipahami oleh algoritma machine learning dan memungkinkan agar dapat melakukan klasifikasi atau analisis lainnya berdasarkan fitur-fitur teks tersebut.






In [ ]:
# Bag of Words (CountVectorizer)
count_vectorizer = CountVectorizer()
count_vector = count_vectorizer.fit_transform(data['processed_text'])

# Term Frequency-Inverse Document Frequency (TfidfVectorizer)
tfidf_vectorizer = TfidfVectorizer()
tfidf_vector = tfidf_vectorizer.fit_transform(data['processed_text'])


## Modelling dengan Naive Bayes dan Random Forest

Data displit menjadi train set dan test set, lalu inisialisasi vectorizer untuk pemodelan agar memakai segala opsi sekaligus

In [ ]:
# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data['processed_text'], data['label'], test_size=0.2, random_state=69)


vectorizers = [
    ('CountVectorizer', CountVectorizer()),
    ('TfidfVectorizer', TfidfVectorizer())
]

Lalu akan dijalankan kode untuk memfitting data pada model

- Dalam setiap iterasi loop vectorizer, data train (X_train) akan diubah menjadi representasi vektor menggunakan fit_transform(). Kemudian, data test (X_test) akan diubah menjadi representasi vektor menggunakan transform(). Representasi vektor ini akan digunakan sebagai fitur dalam model klasifikasi.

- Setelah vectorisasi data, kita memiliki loop untuk setiap model klasifikasi yang diberikan dalam variabel models. Pada setiap iterasi, model_name akan berisi nama model, dan model akan berisi objek model klasifikasi yang sesuai.

- Setelah melatih model, kita dapat melakukan prediksi pada data uji dengan memanggil metode predict() pada objek model dengan menggunakan data uji yang telah divetorisasi (X_test_vectorized). Hasil prediksi disimpan dalam variabel y_pred.

- Dalam loop pertama, dilakukan vectorisasi dengan vectorizer pertama, dan di dalam loop kedua, dilakukan pelatihan dan evaluasi model dengan model pertama. Setelah itu, loop dilakukan untuk vectorizer kedua dan seterusnya, sehingga semua kombinasi vectorizer dan model akan dijalankan.

In [ ]:
for vectorizer_name, vectorizer in vectorizers:
    X_train_vectorized = vectorizer.fit_transform(X_train)
    X_test_vectorized = vectorizer.transform(X_test)

    # Classification models
    models = [
        ('Naive Bayes', MultinomialNB()),
        ('Random Forest Classifier', RandomForestClassifier())
    ]

    for model_name, model in models:
        # Fitting the model
        model.fit(X_train_vectorized, y_train)

        # Predictions
        y_pred = model.predict(X_test_vectorized)

        # Model evaluation
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        print(f'{vectorizer_name} + {model_name}')
        print('Accuracy:', accuracy)
        print(report)
        print('---')

CountVectorizer + Naive Bayes
Accuracy: 0.9432989690721649
                        precision    recall  f1-score   support

                 Books       0.97      0.91      0.94       593
Clothing & Accessories       0.96      0.97      0.97       447
           Electronics       0.93      0.93      0.93       542
             Household       0.92      0.96      0.94       940

              accuracy                           0.94      2522
             macro avg       0.95      0.94      0.95      2522
          weighted avg       0.94      0.94      0.94      2522

---
CountVectorizer + Random Forest Classifier
Accuracy: 0.9389373513084853
                        precision    recall  f1-score   support

                 Books       0.95      0.94      0.95       593
Clothing & Accessories       0.96      0.96      0.96       447
           Electronics       0.96      0.89      0.92       542
             Household       0.91      0.96      0.93       940

              accuracy      

**Recall** mengukur sejauh mana model mampu mengidentifikasi keseluruhan jumlah sampel yang relevan dalam kategori tertentu. Dalam kata lain, recall mengukur kemampuan model untuk mendeteksi semua positif yang sebenarnya. Recall dihitung dengan membagi jumlah positif yang terklasifikasi dengan benar (true positive) oleh jumlah positif yang sebenarnya (true positive + false negative).

**Support** adalah jumlah sampel yang terdapat dalam kategori tertentu dalam data uji. Dalam laporan klasifikasi, support mencerminkan jumlah data yang termasuk dalam setiap kategori yang dievaluasi. Support memberikan gambaran tentang seberapa besar kategori tersebut muncul dalam data uji.


---



---



Berdasarkan hasil klasifikasi, dapat dilihat bahwa penggunaan CountVectorizer memberikan hasil yang sedikit lebih baik daripada TfidfVectorizer dalam hal akurasi dan beberapa metrik evaluasi lainnya. Namun, perbedaannya tidak signifikan. Sedangkan, perbandingan antara Naive Bayes dan Random Forest Classifier menunjukkan bahwa kedua model tersebut memberikan hasil yang serupa dalam kasus ini. Oleh karena itu, pemilihan model tergantung pada preferensi dan kebutuhan spesifik dari masalah klasifikasi yang sedang dihadapi.

# Penggunaan BERT

Pada kasus ini, dapat digunakan suatu model LLM (Large Language Model) untuk membuat klasifikasi yang lebih handal dari model dengan teknik Machine Learning seperti sebelumnya. Pada kasus ini saya menggunakan BERT model yang cukup populer digunakan untuk tujuan klasifikasi. Selain itu pada kasus ini juga harus menggunakan 2 hyparameter tuning.

In [ ]:
!pip install transformers

In [ ]:
import logging
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader
import optuna
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Mengatur level logger ke WARNING untuk menghilangkan peringatan yang tidak perlu
optuna.logging.set_verbosity(optuna.logging.WARNING)
logging.getLogger("transformers").setLevel(logging.WARNING)

### Encoding

`tokenizer = BertTokenizer.from_pretrained('bert-base-uncased'):` Membuat objek tokenizer dengan menggunakan model BERT yang telah di-pretrained. Model yang digunakan dalam contoh ini adalah 'bert-base-uncased', yang merupakan versi BERT dengan huruf-huruf kecil (uncased).

BERT (Bidirectional Encoder Representations from Transformers) adalah salah satu contoh besar dalam kategori Language Model (LM) yang dikembangkan oleh Google pada tahun 2018. Model ini merupakan hasil dari pelatihan secara unsupervised pada tugas-tugas bahasa, yang memungkinkannya untuk memahami dan menghasilkan representasi kata yang kaya dalam konteks kalimat.


Model ini menerima input berupa teks yang sudah di-tokenisasi dan di-encoded, dan menghasilkan prediksi kategori untuk setiap teks.

Selanjutnya dilakukan tokenisasi dan encoding teks menggunakan BERT dengan memanggil metode `batch_encode_plus()` pada tokenizer. Metode ini menghasilkan output berupa input_ids yang merupakan representasi token kata-kata dalam bentuk indeks numerik, attention_mask yang menunjukkan bagian mana dari teks yang harus diperhatikan oleh model, dan beberapa informasi tambahan yang dibutuhkan oleh model BERT.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Mengonversi label kategori menjadi angka
label_mapping = {'Household': 0, 'Books': 1, 'Electronics': 2, 'Clothing & Accessories': 3}
y_train_encoded = [label_mapping[label] for label in y_train]
y_test_encoded = [label_mapping[label] for label in y_test]

# Tokenisasi dan encoding teks menggunakan BERT
X_train_encoded = tokenizer.batch_encode_plus(
    X_train.tolist(),
    truncation=True,
    padding=True,
    return_tensors="pt"
)
X_test_encoded = tokenizer.batch_encode_plus(
    X_test.tolist(),
    truncation=True,
    padding=True,
    return_tensors="pt"
)

# Membuat DataLoader untuk data latih dan data uji
train_dataset = TensorDataset(X_train_encoded['input_ids'].squeeze(1),
                              X_train_encoded['attention_mask'].squeeze(1),
                              torch.tensor(y_train_encoded))
test_dataset = TensorDataset(X_test_encoded['input_ids'].squeeze(1),
                             X_test_encoded['attention_mask'].squeeze(1),
                             torch.tensor(y_test_encoded))

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)


### Tuning Hyperparameter

**Optuna** adalah sebuah framework untuk melakukan optimasi hyperparameter yang memungkinkan pencarian konfigurasi terbaik dari sekumpulan hyperparameter model yang ditentukan sendiri pada kode.

Hyperparameter yang dituning dalam kasus ini adalah learning rate dan weight decay. **Learning rate** adalah parameter yang menentukan seberapa besar langkah yang diambil saat memperbarui bobot model selama pelatihan. **Weight decay** adalah parameter yang mengendalikan kekuatan regularisasi pada bobot model untuk mencegah overfitting.

Pada kode di bawah, tuning dilakukan dengan trials 3 kali dan epoch masing-masing 3 kali, lalu lbrary `tqdm` digunakan untuk printing proses loading epoch

In [ ]:
# Fungsi objektif untuk Optimasi Optuna
def objective(trial):
    # Definisikan hyperparameter yang akan dioptimasi
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)

    # Metode Pre-trained LLM (BERT)
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
    model.to('cuda')

    # Mengoptimasi model menggunakan AdamW optimizer dengan hyperparameter yang dioptimasi
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Training model dengan BERT

    for epoch in range(3):
        model.train()
        train_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}', leave=False)
        for batch in progress_bar:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            _, predicted_labels = torch.max(outputs.logits, dim=1)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)

            progress_bar.set_postfix({'training_loss': train_loss / len(train_dataloader),
                                      'training_accuracy': correct_predictions / total_predictions})

        # Evaluasi hasil klasifikasi dengan BERT pada data uji
        model.eval()
        y_pred_bert = []
        y_true_bert = []

        with torch.no_grad():
            for batch in test_dataloader:
                input_ids, attention_mask, labels = batch
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                outputs = model(input_ids, attention_mask=attention_mask)

                logits = outputs.logits
                _, predicted_labels = torch.max(logits, dim=1)

                y_pred_bert.extend(predicted_labels.tolist())
                y_true_bert.extend(labels.tolist())

        # Mengubah label angka menjadi label teks
        label_mapping = {0: 'Household', 1: 'Books', 2: 'Electronics', 3: 'Clothing & Accessories'}
        y_pred_bert = [label_mapping[pred] for pred in y_pred_bert]
        y_true_bert = [label_mapping[true] for true in y_true_bert]

        # Evaluasi hasil klasifikasi dengan BERT
        accuracy = classification_report(y_true_bert, y_pred_bert, output_dict=True, zero_division=1)['accuracy']
        f1 = f1_score(y_true_bert, y_pred_bert, average='weighted', zero_division=1)


        print(f"Epoch {epoch + 1} - Training Loss: {train_loss / len(train_dataloader)} - Accuracy: {correct_predictions / total_predictions} - F1 Score: {f1}")

    return accuracy

# Fungsi untuk mengeksekusi Optimasi Optuna
def optimize_model():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=3)

    print(f"Best accuracy: {study.best_value}")
    print(f"Best parameters: {study.best_params}")

# Eksekusi Optimasi Optuna
optimize_model()

<ipython-input-15-0c38d9a48ed8>:36: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
<ipython-input-15-0c38d9a48ed8>:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1 - Training Loss: 1.4067259530068956 - Accuracy: 0.3324077746925823 - F1 Score: 0.07603170429204721


Epoch 2 - Training Loss: 1.4053502792223886 - Accuracy: 0.3422253074176914 - F1 Score: 0.20240142093310814


Epoch 3 - Training Loss: 1.4035394319263768 - Accuracy: 0.3340936136453788 - F1 Score: 0.20240142093310814


...

Epoch 3 - Training Loss: 0.0909326334385033 - Accuracy: 0.9777865926219754 - F1 Score: 0.9651176131535377


<ipython-input-15-0c38d9a48ed8>:36: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
<ipython-input-15-0c38d9a48ed8>:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias'

Epoch 1 - Training Loss: 1.3606475091018313 - Accuracy: 0.3696945656485522 - F1 Score: 0.20240142093310814


Epoch 2 - Training Loss: 1.34489420665614 - Accuracy: 0.3853629512098374 - F1 Score: 0.20240142093310814


Epoch 3 - Training Loss: 1.3433464790108465 - Accuracy: 0.3846687822292741 - F1 Score: 0.20240142093310814
Best accuracy: 0.9651070578905631
Best parameters: {'learning_rate': 3.0003086122198434e-05, 'weight_decay': 0.0004268589645496483}


Best accuracy: 0.9651070578905631
Ini adalah akurasi terbaik yang dicapai oleh model dengan konfigurasi hyperparameter terbaik yang ditemukan. Akurasi merupakan metrik yang mengukur sejauh mana model dapat memprediksi label kategori dengan benar. Nilai 0.9651070578905631 menunjukkan bahwa model mencapai akurasi sebesar 96.51% pada data uji.

Best parameters:
- `'learning_rate': 3.0003086122198434e-05, `
- `'weight_decay': 0.0004268589645496483`


Ini adalah kombinasi terbaik dari hyperparameter yang ditemukan oleh Optuna. Dalam kasus ini, nilai terbaik untuk learning rate adalah. Dengan menggunakan kombinasi hyperparameter terbaik yang ditemukan tersebut, kita dapat melatih model klasifikasi dengan konfigurasi yang optimal untuk mencapai kinerja yang lebih baik dalam memprediksi label kategori pada data baru.

### Model dengan Best Hyperparameter

Setelah mendapatkan kombinasi terbaik dari hyperparameter menggunakan Optuna, kita dapat menggunakannya dalam proses training model. Disini model, fungsi training, dsb dideklarasikan ulang agar kode dapat dijalankan walaupun runtime sudah terhapus.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

# Muat best hyperparameter yang ditemukan melalui Optuna
best_hyperparameters = {'learning_rate':  3.0003086122198434e-05, 'weight_decay': 0.0004268589645496483}  # Ganti dengan best hyperparameter Anda

# Set parameter model dengan best hyperparameter
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=best_hyperparameters['learning_rate'],
                              weight_decay=best_hyperparameters['weight_decay'])

# Definisikan fungsi untuk melatih model
def train_model(model, optimizer, train_dataloader):
    model.train()
    train_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch in tqdm(train_dataloader, desc="Training", leave=False):
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, predicted_labels = torch.max(outputs.logits, dim=1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0)

    train_loss /= len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions

    return train_loss, train_accuracy

# Definisikan fungsi untuk melakukan evaluasi model
def evaluate_model(model, test_dataloader):
    model.eval()
    y_pred = []
    y_true = []

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)

            y_pred.extend(predicted_labels.tolist())
            y_true.extend(labels.tolist())

    return y_true, y_pred

# Muat data uji
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Lakukan pelatihan model
num_epochs = 3  # Ganti dengan jumlah epoch yang diinginkan
for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(model, optimizer, train_dataloader)
    print(f"Epoch {epoch+1} - Training Loss: {train_loss:.4f} - Training Accuracy: {train_accuracy:.4f}")

# Evaluasi model menggunakan data uji
y_true, y_pred = evaluate_model(model, test_dataloader)

# Konversi label angka menjadi label teks
label_mapping = {0: 'Household', 1: 'Books', 2: 'Electronics', 3: 'Clothing & Accessories'}
y_true = [label_mapping[label] for label in y_true]
y_pred = [label_mapping[label] for label in y_pred]

# Tampilkan classification report
print(classification_report(y_true, y_pred))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1 - Training Loss: 0.2772 - Training Accuracy: 0.9236


Epoch 2 - Training Loss: 0.1329 - Training Accuracy: 0.9673


Epoch 3 - Training Loss: 0.0820 - Training Accuracy: 0.9800
                        precision    recall  f1-score   support

                 Books       0.99      0.95      0.97       593
Clothing & Accessories       0.98      0.99      0.99       447
           Electronics       0.94      0.96      0.95       542
             Household       0.96      0.97      0.97       940

              accuracy                           0.97      2522
             macro avg       0.97      0.97      0.97      2522
          weighted avg       0.97      0.97      0.97      2522



Dari hasil evaluasi tersebut, dapat dilihat bahwa model memiliki performa yang baik dengan nilai precision, recall, dan f1-score yang tinggi untuk setiap kelas. Selain itu, akurasi secara keseluruhan juga mencapai 98%, menunjukkan kemampuan model dalam memprediksi label kategori dengan benar.

Secara umum model LLM ini telah mendapatkan hasil yang jauh lebih baik dibandingkan dengan teknik machine learning umum (pada kasus ini Naive Bayes dan Random Forest). Namun, konsekuensi dari menggunakan Bert model ini adalah proses trial random search yang memakan waktu sangat lama (1 jam+).